In [158]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [159]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [160]:
data =[]
for i in range(0,1):
    txt1 = open('순종.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '순종' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '국민' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'순종'))

In [161]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("순종실록"," "))
df=df.map(lambda x : x.replace("순종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
len(df)

11

In [162]:
okt = Okt()

In [163]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','모두','우리'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [164]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [165]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


In [166]:
NUM_TOPICS = 15

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	국민	0.019321775063872337
	한국	0.013277055695652962
	조약	0.012859449721872807
	에셔	0.011237877421081066
	우리나라	0.010179594159126282
	관리	0.009192175231873989
	폐하	0.00828863587230444
	일본국	0.008070481941103935
	보호	0.007151174824684858
	일본	0.007027738727629185


Topic ID: 1
	한국	0.010871587321162224
	일본	0.00943248812109232
	국민	0.008751558139920235
	어업	0.008148972876369953
	자기	0.0067577846348285675
	폐하	0.006643646862357855
	우리나라	0.006525431293994188
	협약	0.005085720214992762
	정부	0.004808505065739155
	간도	0.004791170358657837


Topic ID: 2
	우리나라	0.013386867940425873
	일본	0.013026515953242779
	자기	0.008238878101110458
	서로	0.008199808187782764
	폐하	0.008163919672369957
	생각	0.006782235112041235
	스스로	0.006140354089438915
	독립	0.005845020525157452
	국민	0.00576563598588109
	백성	0.005381503142416477


Topic ID: 3
	우리나라	0.01537069957703352
	자기	0.009575898759067059
	폐하	0.007623715791851282
	일본	0.0070467195473611355
	서로	0.007016639690846205
	생각	0.0057763769291341305
	한국	0.0055638

In [167]:
# 주제 :  백성
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.005835 -0.010768       1        1  67.826920
12    -0.115232  0.008373       2        1  14.240247
2      0.051539 -0.001121       3        1  12.090114
7      0.002524  0.039862       4        1   2.257787
4     -0.043415 -0.039613       5        1   1.837113
6      0.037787 -0.002944       6        1   1.192110
9      0.006548  0.002712       7        1   0.359213
1      0.002235  0.018749       8        1   0.052539
0     -0.040147  0.009429       9        1   0.022834
13     0.013868 -0.006986      10        1   0.022137
11     0.007174  0.008546      11        1   0.021078
10    -0.006928 -0.008632      12        1   0.020675
3      0.031390  0.009719      13        1   0.019869
5      0.013839 -0.013956      14        1   0.019294
14     0.032984 -0.013369      15        1   0.018076, topic_info=     Term       Freq      Total Category  logprob  loglift
521    일본  58.000000  58.000000  Default  30.0000  30.0000
769    한국  61.000000  61.000000  Default  29.0000  29.0000
460  우리나라  44.000000  44.000000  Default  28.0000  28.0000
678    폐하  39.000000  39.000000  Default  27.0000  27.0000
81     국민  43.000000  43.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
769    한국   0.004486  61.911903  Topic15  -5.0603  -0.9142
300    부강   0.002441  14.847703  Topic15  -5.6690  -0.0950
81     국민   0.003207  43.342297  Topic15  -5.3958  -0.8931
187    동안   0.001964   9.842213  Topic15  -5.8862   0.0990
63     관리   0.002076  16.943762  Topic15  -5.8309  -0.3890

[998 rows x 6 columns], token_table=      Topic      Freq Term
term                      
986       1  0.530779   가면
833       1  0.830448  가운데
5         1  0.756068   가지
5         2  0.068733   가지
5         3  0.137467   가지
...     ...       ...  ...
1441      1  0.760737   호소
858       1  0.474383   호수
858       2  0.474383   호수
725       1  0.889865   환심
1539      1  0.842198   휴대

[620 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 13, 3, 8, 5, 7, 10, 2, 1, 14, 12, 11, 4, 6, 15])

In [ ]:
# 일본

In [156]:
data =[]
for i in range(0,1):
    txt1 = open('순종.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '순종' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'순종'))
    

In [157]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("순종실록"," "))
df=df.map(lambda x : x.replace("순종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
len(df)

348

In [149]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','모두','우리','부록'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [150]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	이왕직	0.030984675511717796
	일본	0.01894368603825569
	양력	0.01764994114637375
	증보	0.013625402003526688
	대정	0.012220211327075958
	직원	0.011181734502315521
	경선	0.009482117369771004
	부속	0.008458772674202919
	귀족	0.008376304991543293
	물품	0.008372621610760689


Topic ID: 1
	이왕직	0.04923241585493088
	양력	0.03513980656862259
	일본	0.03349738195538521
	대정	0.03277425467967987
	민병석	0.02788432128727436
	총독부	0.024696364998817444
	장관	0.024347465485334396
	천황	0.022882331162691116
	양궁	0.019556310027837753
	당부	0.018705714493989944


Topic ID: 2
	대정	0.032064907252788544
	이왕직	0.030249547213315964
	일본	0.025613145902752876
	양력	0.02441515401005745
	총독부	0.02150706574320793
	대한	0.015816600993275642
	사무관	0.014941351488232613
	당부	0.01427367515861988
	양궁	0.013317510485649109
	천황	0.012310052290558815


Topic ID: 3
	이왕직	0.06258540600538254
	일본	0.028367364779114723
	양력	0.027986256405711174
	대정	0.021635189652442932
	사무관	0.019996795803308487
	장관	0.014676575548946857
	제조	0.009920051321387291

In [151]:
# 주제 :  일본
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16     0.133037 -0.042579       1        1  11.132872
4      0.092542  0.011954       2        1  10.639602
1      0.116043  0.008197       3        1  10.601169
13     0.037369  0.081773       4        1   7.899837
12    -0.143798 -0.175953       5        1   7.316948
17     0.086001 -0.033648       6        1   6.335486
9      0.066003 -0.026007       7        1   6.250012
18     0.017843  0.093364       8        1   5.832780
6      0.111686 -0.006174       9        1   5.423286
3      0.014135 -0.023333      10        1   5.314369
10     0.060632 -0.031837      11        1   5.159800
14     0.034529 -0.112429      12        1   3.949448
15    -0.008628 -0.040157      13        1   3.458026
7      0.006587 -0.003867      14        1   3.307053
2      0.020923  0.136893      15        1   2.529888
0     -0.110958  0.100019      16        1   1.533241
5     -0.144731  0.050874      17        1   1.361066
19    -0.122884  0.058751      18        1   0.812095
8     -0.157921 -0.051534      19        1   0.639139
11    -0.108410  0.005693      20        1   0.503886, topic_info=    Term        Freq       Total Category  logprob  loglift
337   사항  142.000000  142.000000  Default  30.0000  30.0000
154   사무   81.000000   81.000000  Default  29.0000  29.0000
111   제조   80.000000   80.000000  Default  28.0000  28.0000
5    이왕직  827.000000  827.000000  Default  27.0000  27.0000
320   다음   46.000000   46.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
121   관장    0.433958   45.651871  Topic20  -5.3100   0.6347
320   다음    0.428673   46.713928  Topic20  -5.3223   0.5995
192   차관    0.429993   94.613503  Topic20  -5.3192  -0.1032
111   제조    0.406370   80.313606  Topic20  -5.3757   0.0041
50    자작    0.397993  175.179977  Topic20  -5.3965  -0.7966

[1340 rows x 6 columns], token_table=      Topic      Freq Term
term                      
60        1  0.150045   가례
60        4  0.300089   가례
60        5  0.200060   가례
60        7  0.100030   가례
60        9  0.100030   가례
...     ...       ...  ...
269       7  0.153870   히데
269       8  0.038467   히데
269      10  0.076935   히데
269      15  0.038467   히데
953       9  0.583082   히키

[2875 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 5, 2, 14, 13, 18, 10, 19, 7, 4, 11, 15, 16, 8, 3, 1, 6, 20, 9, 12])